# Convert the TensorFlow-Trained BERT Model to a PyTorch Model

In [1]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

In [2]:
%store -r training_job_name

In [3]:
try:
    training_job_name
except NameError:
    print('+++++++++++++++++++++++++++++++')
    print('[ERROR] Please wait for the Training notebook to finish.')
    print('+++++++++++++++++++++++++++++++')

In [4]:
print('Previous training_job_name: {}'.format(training_job_name))

Previous training_job_name: tensorflow-training-2020-09-26-18-44-25-975


# Download the TensorFlow-Trained Model from S3 to this Notebook

In [5]:
models_dir = './models'

In [6]:
!aws s3 cp s3://$bucket/$training_job_name/output/model.tar.gz $models_dir/model.tar.gz

download: s3://sagemaker-us-west-2-085964654406/tensorflow-training-2020-09-26-18-44-25-975/output/model.tar.gz to models/model.tar.gz


In [7]:
import tarfile
import pickle as pkl

try:
    tar = tarfile.open('{}/model.tar.gz'.format(models_dir))
    tar.extractall(path=models_dir)
    tar.close()
except Exception as e:
    print('[ERROR] in tar operation: {}'.format(e))

In [8]:
!ls -al $models_dir

total 825188
drwxrwxr-x  7 ec2-user ec2-user      4096 Sep 26 19:20 .
drwxrwxr-x 11 ec2-user ec2-user      4096 Sep 26 19:20 ..
drwxr-xr-x  2 ec2-user ec2-user      4096 Sep 26 19:17 code
drwxr-xr-x  2 ec2-user ec2-user      4096 Sep 26 19:17 metrics
-rw-rw-r--  1 ec2-user ec2-user 844962197 Sep 26 19:18 model.tar.gz
drwxr-xr-x  2 ec2-user ec2-user      4096 Sep 26 18:47 tensorboard
drwxr-xr-x  3 ec2-user ec2-user      4096 Sep 26 18:47 tensorflow
drwxr-xr-x  3 ec2-user ec2-user      4096 Sep 26 18:47 transformers


In [9]:
transformer_model_dir = '{}/transformers/fine-tuned/'.format(models_dir)

!ls -al $transformer_model_dir

total 261692
drwxr-xr-x 2 ec2-user ec2-user      4096 Sep 26 19:16 .
drwxr-xr-x 3 ec2-user ec2-user      4096 Sep 26 18:47 ..
-rw-r--r-- 1 ec2-user ec2-user      1358 Sep 26 19:16 config.json
-rw-r--r-- 1 ec2-user ec2-user 267959068 Sep 26 19:16 tf_model.h5


In [10]:
cat $transformer_model_dir/config.json

{
  "_num_labels": 5,
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "bad_words_ids": null,
  "bos_token_id": null,
  "decoder_start_token_id": null,
  "dim": 768,
  "do_sample": false,
  "dropout": 0.1,
  "early_stopping": false,
  "eos_token_id": null,
  "finetuning_task": null,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embeddings": 512,
  "min_length": 0,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "no_repeat_ngram_size": 0,
  "num_beams": 1,
  "num_return_sequences": 1,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true

# Convert the TensorFlow Model to PyTorch

In [11]:
from transformers import DistilBertForSequenceClassification # PyTorch version

try:
    loaded_pytorch_model = DistilBertForSequenceClassification.from_pretrained(transformer_model_dir,
                                                                         id2label={
                                                                           0: 1,
                                                                           1: 2,
                                                                           2: 3,
                                                                           3: 4,
                                                                           4: 5
                                                                         },
                                                                         label2id={
                                                                           1: 0,
                                                                           2: 1,
                                                                           3: 2,
                                                                           4: 3,
                                                                           5: 4
                                                                         },
                                                                      from_tf=True)
except Exception as e:
    print('[ERROR] in loading model {}: '.format(e))

In [12]:
print(type(loaded_pytorch_model))
print(loaded_pytorch_model)

<class 'transformers.modeling_distilbert.DistilBertForSequenceClassification'>
DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
   

# Predict with the PyTorch Model

In [13]:
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

inference_device = -1 # CPU: -1, GPU: 0
print('inference_device {}'.format(inference_device))

inference_device -1


In [14]:
from transformers import TextClassificationPipeline

inference_pipeline = TextClassificationPipeline(model=loaded_pytorch_model, 
                                                tokenizer=tokenizer,
                                                framework='pt',
                                                device=inference_device) 

In [15]:
review = """I loved it!  I will recommend this to everyone."""
print(review, inference_pipeline(review))

I loved it!  I will recommend this to everyone. [{'label': 5, 'score': 0.94240725}]


In [16]:
review = """Really bad.  I hope they don't make this anymore."""
print(review, inference_pipeline(review))

Really bad.  I hope they don't make this anymore. [{'label': 1, 'score': 0.9148224}]


# Save The Transformer/PyTorch Model with `.save_pretrained()`

In [17]:
pytorch_models_dir = './models/transformers/pytorch'

In [18]:
!mkdir -p $pytorch_models_dir

In [19]:
loaded_pytorch_model.save_pretrained(pytorch_models_dir)

In [20]:
!ls -al $pytorch_models_dir

total 261588
drwxrwxr-x 2 ec2-user ec2-user      4096 Sep 26 19:20 .
drwxr-xr-x 4 ec2-user ec2-user      4096 Sep 26 19:20 ..
-rw-rw-r-- 1 ec2-user ec2-user      1302 Sep 26 19:20 config.json
-rw-rw-r-- 1 ec2-user ec2-user 267852933 Sep 26 19:20 pytorch_model.bin


# Load and Predict

In [21]:
pytorch_model = DistilBertForSequenceClassification.from_pretrained(pytorch_models_dir)

In [22]:
from transformers import TextClassificationPipeline

inference_pipeline = None

try: 
    inference_pipeline = TextClassificationPipeline(model=pytorch_model, 
                                                    tokenizer=tokenizer,
                                                    framework='pt',
                                                    device=inference_device) 
except Exception as e:
    print('[ERROR] in creating inference_pipeline: {}'.format(e))

In [23]:
review = """I loved it!  I will recommend this to everyone."""
print(review, inference_pipeline(review))

I loved it!  I will recommend this to everyone. [{'label': 5, 'score': 0.94240725}]


# Upload Transformer/PyTorch Model to S3

In [24]:
transformer_pytorch_model_name = 'pytorch_model.bin'

In [25]:
transformer_pytorch_model_s3_uri = 's3://{}/models/transformer-pytorch/'.format(bucket)
print(transformer_pytorch_model_s3_uri)

s3://sagemaker-us-west-2-085964654406/models/transformer-pytorch/


In [26]:
!aws s3 cp --recursive $pytorch_models_dir $transformer_pytorch_model_s3_uri

upload: models/transformers/pytorch/config.json to s3://sagemaker-us-west-2-085964654406/models/transformer-pytorch/config.json
upload: models/transformers/pytorch/pytorch_model.bin to s3://sagemaker-us-west-2-085964654406/models/transformer-pytorch/pytorch_model.bin


In [27]:
!aws s3 ls $transformer_pytorch_model_s3_uri

2020-09-26 19:20:38       1302 config.json
2020-09-26 19:20:38  267852933 pytorch_model.bin


In [28]:
%store transformer_pytorch_model_name

Stored 'transformer_pytorch_model_name' (str)


In [29]:
%store transformer_pytorch_model_s3_uri

Stored 'transformer_pytorch_model_s3_uri' (str)


In [30]:
%store

Stored variables and their in-db values:
auto_ml_job_name                                      -> 'automl-dm-26-16-00-25'
autopilot_endpoint_name                               -> 'automl-dm-ep-26-16-21-49'
autopilot_train_s3_uri                                -> 's3://sagemaker-us-west-2-085964654406/data/amazon
balance_dataset                                       -> True
experiment_name                                       -> 'Amazon-Customer-Reviews-BERT-Experiment-160114585
ingest_create_athena_db_passed                        -> True
ingest_create_athena_table_parquet_passed             -> True
ingest_create_athena_table_tsv_passed                 -> True
max_seq_length                                        -> 64
prepare_trial_component_name                          -> 'TrialComponent-2020-09-26-184417-oahc'
processed_test_data_s3_uri                            -> 's3://sagemaker-us-west-2-085964654406/sagemaker-s
processed_train_data_s3_uri                           -> 's3://sa

In [ ]:
%%javascript
Jupyter.notebook.save_checkpoint();
Jupyter.notebook.session.delete();